In [16]:
from backend_models import run_poisson, run_nb, run_nbp, df


## Parámetros del modelo


In [26]:
import ipywidgets as widgets
from IPython.display import display

# Selector de modelo
model_selector = widgets.Dropdown(
    options=['Poisson', 'Negative Binomial', 'NB-P'],
    description='Modelo:',
    style={'description_width':'initial'}
)

# Hiperparámetros
chains_widget = widgets.BoundedIntText(
    value=3, min=1, max=10, step=1,
    description='Cadenas:',
    style={'description_width':'initial'}
)
warmup_widget = widgets.BoundedIntText(
    value=1000, min=0, max=50000, step=100,
    description='Warmup:',
    style={'description_width':'initial'}
)
sample_widget = widgets.BoundedIntText(
    value=2000, min=0, max=50000, step=100,
    description='Sampling:',
    style={'description_width':'initial'}
)

# Botón para lanzar el ajuste
run_button = widgets.Button(
    description='Ejecutar modelo',
    button_style='primary',
    tooltip='Lanza el ajuste con los parámetros seleccionados'
)

# Muestra los widgets
display(widgets.VBox([
    model_selector,
    widgets.HBox([chains_widget, warmup_widget, sample_widget]),
    run_button
]))


DEBUG:cmdstanpy:cmd: /content/pois_model info
cwd: None
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyfa6aja6/waflxs80.json
03:35:07 - cmdstanpy - INFO - CmdStan start processing
INFO:cmdstanpy:CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/pois_model', 'id=1', 'random', 'seed=33221', 'data', 'file=/tmp/tmpyfa6aja6/waflxs80.json', 'output', 'file=/tmp/tmpyfa6aja6/pois_modelqme_venu/pois_model-20250612033507_1.csv', 'method=sample', 'num_samples=2000', 'num_warmup=1000', 'thin=1', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 1
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/pois_model', 'id=2', 'random', 'seed=33221', 'data', 'file=/tmp/tmpyfa6aja6/waflxs80.json', 'output', 'file=/tmp/tmpyfa6aja6/pois_modelqme_venu/pois_model-20250612033507_2.csv', 'method=sample', 'num_samples=2000', 'num_warmup=1000', 'thin=1', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 2
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/pois_model', 'id=3', 'random', 'seed=33221', 'data', 'file=/tmp/tmpyfa6aja6/waflxs80.json', 'output',

03:35:23 - cmdstanpy - INFO - CmdStan done processing.
INFO:cmdstanpy:CmdStan done processing.
DEBUG:cmdstanpy:runset
RunSet: chains=3, chain_ids=[1, 2, 3], num_processes=3
 cmd (chain 1):
	['/content/pois_model', 'id=1', 'random', 'seed=33221', 'data', 'file=/tmp/tmpyfa6aja6/waflxs80.json', 'output', 'file=/tmp/tmpyfa6aja6/pois_modelqme_venu/pois_model-20250612033507_1.csv', 'method=sample', 'num_samples=2000', 'num_warmup=1000', 'thin=1', 'algorithm=hmc', 'adapt', 'engaged=1']
 retcodes=[0, 0, 0]
 per-chain output files (showing chain 1 only):
 csv_file:
	/tmp/tmpyfa6aja6/pois_modelqme_venu/pois_model-20250612033507_1.csv
 console_msgs (if any):
	/tmp/tmpyfa6aja6/pois_modelqme_venu/pois_model-20250612033507_0-stdout.txt
DEBUG:cmdstanpy:Chain 1 console:
method = sample (Default)
  sample
    num_samples = 2000
    num_warmup = 1000 (Default)
    save_warmup = false (Default)
    thin = 1 (Default)
    adapt
      engaged = true (Default)
      gamma = 0.05 (Default)
      delta = 0.8 

### Ejecución de modelos

In [27]:
import ipywidgets as widgets
from IPython.display import display

# 3 contenedores separados
output_summary   = widgets.Output(layout={'border':'1px solid gray','padding':'5px'})
output_histogram = widgets.Output(layout={'border':'1px solid gray','padding':'5px'})
output_plot      = widgets.Output(layout={'border':'1px solid gray','padding':'5px'})

# Muestra los contenedores con sus títulos
display(widgets.VBox([
    widgets.HTML('<b>Summary</b>'), output_summary,
    widgets.HTML('<b>Histogramas</b>'), output_histogram,
    widgets.HTML('<b>Ajuste vs Observados</b>'), output_plot
]))


In [28]:
def on_run_clicked(_):
    import numpy as np
    import matplotlib.pyplot as plt
    from IPython.display import display, clear_output

    mdl    = model_selector.value
    chains = chains_widget.value
    warmup = warmup_widget.value
    sample = sample_widget.value

    # 1) Limpiar cada salida
    for out in (output_summary, output_histogram, output_plot):
        out.clear_output()

    # 2) Ajustar el modelo
    if mdl == 'Poisson':
        fit = run_poisson(chains=chains, warmup=warmup, sample=sample)
    elif mdl == 'Negative Binomial':
        fit = run_nb(chains=chains, warmup=warmup, sample=sample)
    else:
        fit = run_nbp(chains=chains, warmup=warmup, sample=sample)

    # 3) Summary
    with output_summary:
        smry = fit.summary()
        if mdl == 'Poisson':
            params = ['beta[1]', 'beta[2]']
        elif mdl == 'Negative Binomial':
            params = ['beta[1]', 'beta[2]', 'theta', 'Dispersion']
        else:  # NB-P
            params = ['beta[1]', 'beta[2]', 'theta', 'Q', 'dispersion']
        # Selecciona solo esas filas y las columnas que quieras
        cols = ['Mean','StdDev','5%','95%','R_hat']
        display(smry.loc[params, cols])


    # 4) Histogramas
    with output_histogram:
        # Datos comunes
        x_obs = df['MV_T'].values
        y_obs = df['N_GC'].values
        markers = ['o','s','^','d','x']
        type_obs = df['Type'].values if 'Type' in df.columns else None

        def plot_hist(data, title, ax):
            ax.hist(data, bins=30, color='grey', edgecolor='black')
            ax.set_title(title)
            ax.set_xlabel('Valor')
            ax.set_ylabel('Frecuencia')

        fig, axes = plt.subplots(
            1 if mdl=='Poisson' else (2 if mdl=='Negative Binomial' else 3),
            (3 if mdl=='Poisson' else 2),
            figsize=(12, 4 if mdl=='Poisson' else (8,12)[mdl!='Poisson'])
        )
        axes = np.array(axes).flatten()

        if mdl == 'Poisson':
            beta_draws = fit.stan_variable('beta')
            mu_draws   = fit.stan_variable('mu')
            N, K       = mu_draws.shape[1], beta_draws.shape[1]
            plot_hist(beta_draws[:,0], r'$\beta_0$', axes[0])
            plot_hist(beta_draws[:,1], r'$\beta_1$', axes[1])
            disp = ((y_obs - mu_draws)**2 / mu_draws).sum(axis=1)/(N-K)
            plot_hist(disp, 'Dispersión', axes[2])

        elif mdl == 'Negative Binomial':
            sims_nb = fit.draws_pd()
            for ax,var in zip(axes, ['beta[1]','beta[2]','theta','Dispersion']):
                plot_hist(sims_nb[var], f'{var}', ax)

        else:  # NB-P
            sims_nbp = fit.draws_pd()
            for ax,var in zip(axes, ['dispersion','Q','beta[1]','beta[2]','theta']):
                plot_hist(sims_nbp[var], f'{var}', ax)

        plt.tight_layout()
        plt.show()

    # 5) Ajuste vs Observados
    with output_plot:
        # datos básicos
        x_obs = df['MV_T'].values
        y_obs = df['N_GC'].values

        x_seq = np.linspace(x_obs.min(), x_obs.max(), 100)
        markers = ['o','s','^','d','x']
        type_obs = df['Type'].values if 'Type' in df.columns else None

        plt.figure(figsize=(8,6))

        if mdl == 'Poisson':
            beta_draws = fit.stan_variable('beta')
            n_draw     = beta_draws.shape[0]
            pred = np.zeros((n_draw, len(x_seq)))
            for i in range(n_draw):
                mu = np.exp(beta_draws[i,0] + beta_draws[i,1]*x_seq)
                pred[i,:] = np.random.poisson(mu)
            l95 = np.percentile(pred,2.5,axis=0)
            u95 = np.percentile(pred,97.5,axis=0)
            m50 = np.percentile(pred,50,axis=0)
            plt.fill_between(x_seq, l95, u95, alpha=0.3, color='lightblue')
            plt.plot(x_seq, m50, '--k', lw=2)

        elif mdl == 'Negative Binomial':
            beta_draws  = fit.stan_variable('beta')
            theta_draws = fit.stan_variable('theta')
            n_draw = beta_draws.shape[0]
            pred = np.zeros((n_draw, len(x_seq)))
            for i in range(n_draw):
                mu = np.exp(beta_draws[i,0] + beta_draws[i,1]*x_seq)
                p  = theta_draws[i]/(theta_draws[i]+mu)
                pred[i,:] = np.random.negative_binomial(theta_draws[i], p)
            l95 = np.percentile(pred,2.5,axis=0)
            u95 = np.percentile(pred,97.5,axis=0)
            m50 = np.percentile(pred,50,axis=0)
            plt.fill_between(x_seq, l95, u95, alpha=0.3, color='lightblue')
            plt.plot(x_seq, m50, '--k', lw=2)

        else:  # NB-P
            beta_draws  = fit.stan_variable('beta')
            theta_draws = fit.stan_variable('theta')
            Q_draws     = fit.stan_variable('Q')
            n_draw = beta_draws.shape[0]
            pred = np.zeros((n_draw, len(x_seq)))
            for i in range(n_draw):
                mu      = np.exp(beta_draws[i,0] + beta_draws[i,1]*x_seq)
                phi_eff = theta_draws[i]*mu**Q_draws[i]
                lam     = np.random.gamma(phi_eff, mu/phi_eff, size=len(x_seq))
                pred[i,:] = np.random.poisson(lam)
            l95 = np.percentile(pred,2.5,axis=0)
            u95 = np.percentile(pred,97.5,axis=0)
            m50 = np.percentile(pred,50,axis=0)
            plt.fill_between(x_seq, l95, u95, alpha=0.3, color='lightblue')
            plt.plot(x_seq, m50, '--k', lw=2)

        # puntos observados
        if type_obs is not None:
            for t,mk in zip(np.unique(type_obs), markers):
                mask = (type_obs==t)
                plt.scatter(x_obs[mask], y_obs[mask], label=str(t), marker=mk)
        else:
            plt.scatter(x_obs, y_obs, label='Obs')

        plt.yscale('log')
        plt.xlabel('MV_T'); plt.ylabel('N_GC')
        plt.legend(title='Type'); plt.tight_layout(); plt.show()

# reengancha el botón
run_button.on_click(on_run_clicked)
